# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [1]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('c:/users/14372/scaling_to_production/05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [2]:
# Separate the features (X) and the target variable (Y)
X = adult_dt.drop(columns=['income'])  # Drop the target variable from the features DataFrame
Y = adult_dt[['income']]  # Target variable as a DataFrame

# Convert categorical features to numerical values using one-hot encoding
X = pd.get_dummies(X, drop_first=True)

# Display the first few rows of the features DataFrame and target DataFrame
print("Features DataFrame (X):")
print(X.head())
print("\nTarget DataFrame (Y):")
print(Y.head())

# Optional: Display the shapes of the DataFrames
print("\nShape of X:", X.shape)
print("Shape of Y:", Y.shape)

Features DataFrame (X):
   age  fnlwgt  education-num  capital-gain  capital-loss  hours-per-week  \
0   39   77516             13          2174             0              40   
1   50   83311             13             0             0              13   
2   38  215646              9             0             0              40   
3   53  234721              7             0             0              40   
4   28  338409             13             0             0              40   

   workclass_ Federal-gov  workclass_ Local-gov  workclass_ Never-worked  \
0                   False                 False                    False   
1                   False                 False                    False   
2                   False                 False                    False   
3                   False                 False                    False   
4                   False                 False                    False   

   workclass_ Private  ...  native-country_ Portugal  \


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

The random_state of the splitting function is a fixed seed value used by the random number generator to ensure that the train-test split of the data is consistent across different runs of the code. It is useful because it ensures reproducibility of results, aids in debugging by providing a consistent dataset split for each run, and helps in documenting the experimental setup. By setting the random_state, i will be to ensure that my experiments are fair and that my results can be replicated by others.

*(Comment here.)*

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier


numerical_features = ['num_col1', 'num_col2', 'num_col3']
categorical_features = ['cat_col1', 'cat_col2']

# Define the preprocessing for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

# Define the preprocessing for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

# Define the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Define the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # The ColumnTransformer from the previous section
    ('classifier', RandomForestClassifier())  # The RandomForestClassifier
])

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# Define features
numerical_features = ['num_col1', 'num_col2', 'num_col3']
categorical_features = ['cat_col1', 'cat_col2']

# Define transformers for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='error', drop='if_binary'))
])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Optional: Visualize the column transformer structure
from sklearn import set_config
set_config(display='diagram')
preprocessor


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('scaler', RobustScaler())]),
                                 ['num_col1', 'num_col2', 'num_col3']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='if_binary'))]),
                                 ['cat_col1', 'cat_col2'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Define features
numerical_features = ['num_col1', 'num_col2', 'num_col3']
categorical_features = ['cat_col1', 'cat_col2']

# Define transformers for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Create the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Optional: Visualize the pipeline structure
from sklearn import set_config
set_config(display='diagram')
model_pipeline


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['num_col1', 'num_col2',
                                                   'num_col3']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['cat_col1', 'cat_col2'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [9]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

# Define the column names
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]

# Load the dataset
adult_dt = (pd.read_csv('c:/users/14372/scaling_to_production/05_src/data/adult/adult.data', header=None, names=columns)
            .assign(income=lambda x: (x.income.str.strip() == '>50K') * 1))

# Separate the features (X) and the target variable (Y)
X = adult_dt.drop(columns=['income'])  # Drop the target variable from the features DataFrame
Y = adult_dt[['income']]  # Target variable as a DataFrame

# Identify numerical and categorical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Define the preprocessing for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

# Define the preprocessing for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

# Define the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Define the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # The ColumnTransformer from the previous section
    ('classifier', RandomForestClassifier(random_state=42))  # The RandomForestClassifier
])

# Define the performance metrics
scoring = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy']

# Perform cross-validation
cv_results = cross_validate(
    model_pipeline, 
    X, 
    Y.values.ravel(),  # Flatten Y to match the expected shape
    cv=5,  # Five-fold cross-validation
    scoring=scoring,
    return_train_score=True  # Include training scores for comparison
)

# Display the results
print("\nCross-Validation Results:")
print(f"Training Negative Log Loss: {cv_results['train_neg_log_loss'].mean():.4f} ± {cv_results['train_neg_log_loss'].std():.4f}")
print(f"Validation Negative Log Loss: {cv_results['test_neg_log_loss'].mean():.4f} ± {cv_results['test_neg_log_loss'].std():.4f}")
print(f"Training ROC AUC: {cv_results['train_roc_auc'].mean():.4f} ± {cv_results['train_roc_auc'].std():.4f}")
print(f"Validation ROC AUC: {cv_results['test_roc_auc'].mean():.4f} ± {cv_results['test_roc_auc'].std():.4f}")
print(f"Training Accuracy: {cv_results['train_accuracy'].mean():.4f} ± {cv_results['train_accuracy'].std():.4f}")
print(f"Validation Accuracy: {cv_results['test_accuracy'].mean():.4f} ± {cv_results['test_accuracy'].std():.4f}")
print(f"Training Balanced Accuracy: {cv_results['train_balanced_accuracy'].mean():.4f} ± {cv_results['train_balanced_accuracy'].std():.4f}")
print(f"Validation Balanced Accuracy: {cv_results['test_balanced_accuracy'].mean():.4f} ± {cv_results['test_balanced_accuracy'].std():.4f}")


c:\Users\14372\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\14372\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(



Cross-Validation Results:
Training Negative Log Loss: -0.0806 ± 0.0005
Validation Negative Log Loss: -0.3751 ± 0.0129
Training ROC AUC: 1.0000 ± 0.0000
Validation ROC AUC: 0.9023 ± 0.0032
Training Accuracy: 1.0000 ± 0.0000
Validation Accuracy: 0.8547 ± 0.0027
Training Balanced Accuracy: 1.0000 ± 0.0000
Validation Balanced Accuracy: 0.7754 ± 0.0049


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [10]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

# Define the column names
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]

# Load the dataset
adult_dt = (pd.read_csv('c:/users/14372/scaling_to_production/05_src/data/adult/adult.data', header=None, names=columns)
            .assign(income=lambda x: (x.income.str.strip() == '>50K') * 1))

# Separate the features (X) and the target variable (Y)
X = adult_dt.drop(columns=['income'])  # Drop the target variable from the features DataFrame
Y = adult_dt[['income']]  # Target variable as a DataFrame

# Identify numerical and categorical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Define the preprocessing for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

# Define the preprocessing for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

# Define the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Define the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # The ColumnTransformer from the previous section
    ('classifier', RandomForestClassifier(random_state=42))  # The RandomForestClassifier
])

# Define the performance metrics
scoring = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy']

# Perform cross-validation
cv_results = cross_validate(
    model_pipeline, 
    X, 
    Y.values.ravel(),  # Flatten Y to match the expected shape
    cv=5,  # Five-fold cross-validation
    scoring=scoring,
    return_train_score=True  # Include training scores for comparison
)

# Convert cross-validation results to a DataFrame
fold_results = pd.DataFrame({
    'fold': range(1, 6),
    'train_neg_log_loss': cv_results['train_neg_log_loss'],
    'test_neg_log_loss': cv_results['test_neg_log_loss'],
    'train_roc_auc': cv_results['train_roc_auc'],
    'test_roc_auc': cv_results['test_roc_auc'],
    'train_accuracy': cv_results['train_accuracy'],
    'test_accuracy': cv_results['test_accuracy'],
    'train_balanced_accuracy': cv_results['train_balanced_accuracy'],
    'test_balanced_accuracy': cv_results['test_balanced_accuracy']
})

# Sort the results by the negative log loss of the test (validation) set
sorted_results = fold_results.sort_values(by='test_neg_log_loss')

# Display the sorted fold-level results
print("Fold-Level Cross-Validation Results:")
print(sorted_results)


c:\Users\14372\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\14372\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Fold-Level Cross-Validation Results:
   fold  train_neg_log_loss  test_neg_log_loss  train_roc_auc  test_roc_auc  \
1     2           -0.079882          -0.395460            1.0      0.896845   
0     1           -0.080468          -0.379042            1.0      0.902153   
4     5           -0.081341          -0.375717            1.0      0.906950   
2     3           -0.080695          -0.369435            1.0      0.902365   
3     4           -0.080591          -0.356018            1.0      0.903304   

   train_accuracy  test_accuracy  train_balanced_accuracy  \
1        0.999962       0.851966                 0.999975   
0        0.999962       0.853677                 0.999920   
4        0.999962       0.857647                 0.999975   
2        0.999962       0.852119                 0.999975   
3        1.000000       0.858108                 1.000000   

   test_balanced_accuracy  
1                0.769466  
0                0.770268  
4                0.782353  
2        

Calculate the mean of each metric. 

In [11]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

# Define the column names
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]

# Load the dataset
adult_dt = (pd.read_csv('c:/users/14372/scaling_to_production/05_src/data/adult/adult.data', header=None, names=columns)
            .assign(income=lambda x: (x.income.str.strip() == '>50K') * 1))

# Separate the features (X) and the target variable (Y)
X = adult_dt.drop(columns=['income'])  # Drop the target variable from the features DataFrame
Y = adult_dt[['income']]  # Target variable as a DataFrame

# Identify numerical and categorical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Define the preprocessing for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

# Define the preprocessing for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

# Define the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Define the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # The ColumnTransformer from the previous section
    ('classifier', RandomForestClassifier(random_state=42))  # The RandomForestClassifier
])

# Define the performance metrics
scoring = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy']

# Perform cross-validation
cv_results = cross_validate(
    model_pipeline, 
    X, 
    Y.values.ravel(),  # Flatten Y to match the expected shape
    cv=5,  # Five-fold cross-validation
    scoring=scoring,
    return_train_score=True  # Include training scores for comparison
)

# Calculate the mean and standard deviation for each metric
metrics_summary = {
    'Metric': ['train_neg_log_loss', 'test_neg_log_loss', 'train_roc_auc', 'test_roc_auc', 'train_accuracy', 'test_accuracy', 'train_balanced_accuracy', 'test_balanced_accuracy'],
    'Mean': [
        cv_results['train_neg_log_loss'].mean(),
        cv_results['test_neg_log_loss'].mean(),
        cv_results['train_roc_auc'].mean(),
        cv_results['test_roc_auc'].mean(),
        cv_results['train_accuracy'].mean(),
        cv_results['test_accuracy'].mean(),
        cv_results['train_balanced_accuracy'].mean(),
        cv_results['test_balanced_accuracy'].mean()
    ],
    'Standard Deviation': [
        cv_results['train_neg_log_loss'].std(),
        cv_results['test_neg_log_loss'].std(),
        cv_results['train_roc_auc'].std(),
        cv_results['test_roc_auc'].std(),
        cv_results['train_accuracy'].std(),
        cv_results['test_accuracy'].std(),
        cv_results['train_balanced_accuracy'].std(),
        cv_results['test_balanced_accuracy'].std()
    ]
}

metrics_df = pd.DataFrame(metrics_summary)

# Display the metrics DataFrame
print("Performance Metrics Summary:")
print(metrics_df)


c:\Users\14372\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\14372\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Performance Metrics Summary:
                    Metric      Mean  Standard Deviation
0       train_neg_log_loss -0.080596        4.671101e-04
1        test_neg_log_loss -0.375134        1.285828e-02
2            train_roc_auc  1.000000        1.191004e-07
3             test_roc_auc  0.902323        3.237501e-03
4           train_accuracy  0.999969        1.535582e-05
5            test_accuracy  0.854704        2.664059e-03
6  train_balanced_accuracy  0.999969        2.620138e-05
7   test_balanced_accuracy  0.775385        4.889208e-03


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [12]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, balanced_accuracy_score, log_loss
from sklearn.model_selection import train_test_split

# Define the column names
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]

# Load the dataset
adult_dt = (pd.read_csv('c:/users/14372/scaling_to_production/05_src/data/adult/adult.data', header=None, names=columns)
            .assign(income=lambda x: (x.income.str.strip() == '>50K') * 1))

# Separate the features (X) and the target variable (Y)
X = adult_dt.drop(columns=['income'])  # Drop the target variable from the features DataFrame
Y = adult_dt[['income']]  # Target variable as a DataFrame

# Identify numerical and categorical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Define the preprocessing for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

# Define the preprocessing for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

# Define the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Define the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # The ColumnTransformer from the previous section
    ('classifier', RandomForestClassifier(random_state=42))  # The RandomForestClassifier
])

# Separate the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Fit the model pipeline on the training data
model_pipeline.fit(X_train, Y_train.values.ravel())  # Flatten Y to match the expected shape

# Make predictions on the test data
Y_pred_proba = model_pipeline.predict_proba(X_test)  # Get the probability scores for the positive class
Y_pred = model_pipeline.predict(X_test)  # Get the class predictions

# Calculate performance metrics
roc_auc = roc_auc_score(Y_test, Y_pred_proba[:, 1])
accuracy = accuracy_score(Y_test, Y_pred)
balanced_accuracy = balanced_accuracy_score(Y_test, Y_pred)
neg_log_loss = log_loss(Y_test, Y_pred_proba)

# Create a dictionary to hold the results
metrics_results = {
    'Negative Log Loss': neg_log_loss,
    'ROC AUC': roc_auc,
    'Accuracy': accuracy,
    'Balanced Accuracy': balanced_accuracy
}

# Display the results
print("Performance Metrics on Test Data:")
print(metrics_results)


Performance Metrics on Test Data:
{'Negative Log Loss': 0.39726506589398913, 'ROC AUC': 0.8999828704291436, 'Accuracy': 0.8551540587572934, 'Balanced Accuracy': 0.7751631656838177}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

Recoding the target variable income while loading the data is convenient because it simplifies the code, improves efficiency by performing the transformation in a vectorized manner, prepares the data for machine learning tasks by converting categorical values into a numerical format, and integrates seamlessly into the data loading process, ensuring consistency and readiness for subsequent analysis or modeling steps. This approach reduces the need for additional data manipulation steps and streamlines the data preparation workflow.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.